In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/aliannejadi/ClariQ

Cloning into 'ClariQ'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 220 (delta 19), reused 32 (delta 14), pack-reused 176
Receiving objects: 100% (220/220), 253.36 MiB | 17.63 MiB/s, done.
Resolving deltas: 100% (110/110), done.
Updating files: 100% (41/41), done.


In [ ]:
!mkdir data
!mkdir data/clariq
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/dev.tsv
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/train.tsv
!cd data/clariq; wget https://github.com/aliannejadi/ClariQ/raw/master/data/question_bank.tsv
!mv data/clariq/train.tsv data/clariq/train_original.tsv

--2023-04-15 07:00:03--  https://github.com/aliannejadi/ClariQ/raw/master/data/dev.tsv
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aliannejadi/ClariQ/master/data/dev.tsv [following]
--2023-04-15 07:00:03--  https://raw.githubusercontent.com/aliannejadi/ClariQ/master/data/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 585998 (572K) [text/plain]
Saving to: ‘dev.tsv’

dev.tsv             100%[===================>] 572.26K  --.-KB/s    in 0.004s  

2023-04-15 07:00:04 (142 MB/s) - ‘dev.tsv’ saved [585998/585998]

--2023-04-15 07:00:04--  https://github.com/aliannejadi/ClariQ/raw/mas

In [ ]:
import pandas as pd
data_path = "./data/"

train = pd.read_csv(data_path+"clariq/train_original.tsv", sep="\t")
valid = pd.read_csv(data_path+"clariq/dev.tsv", sep="\t")

train = train[["initial_request", "question"]]
train.columns = ["query", "clarifying_question"]
train = train[~train["clarifying_question"].isnull()]

valid = valid[["initial_request", "question"]]
valid.columns = ["query", "clarifying_question"]
valid = valid[~valid["clarifying_question"].isnull()]

train.to_csv(data_path+"clariq/train.tsv", sep="\t", index=False)
valid.to_csv(data_path+"clariq/valid.tsv", sep="\t", index=False)

In [ ]:
question_bank = pd.read_csv(data_path+"clariq/question_bank.tsv", sep="\t")
question_bank.head()

,question_id,question
0,Q00001,NaN
1,Q00002,a total cholesterol of 180 to 200 mgdl 10 to 1...
2,Q00003,about how many years experience do you want th...
3,Q00004,according to anima the bible or what other source
4,Q00005,ae you looking for examples of septic system d...


In [ ]:
! pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Files paths
request_file_path = '/content/ClariQ/data/dev.tsv'
question_bank_path = '/content/ClariQ/data/question_bank.tsv'
run_file_path = 'dev_bm25'

import pandas as pd
from rank_bm25 import BM25Okapi
import nltk
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

def stem_tokenize(text, remove_stopwords=True):
  stemmer = PorterStemmer()
  tokens = [word for sent in nltk.sent_tokenize(text) \
                                      for word in nltk.word_tokenize(sent)]
  tokens = [word for word in tokens if word not in \
          nltk.corpus.stopwords.words('english')]
  return [stemmer.stem(word) for word in tokens]

# Reads files and build bm25 corpus (index)
dev = pd.read_csv(request_file_path, sep='\t')
question_bank = pd.read_csv(question_bank_path, sep='\t').fillna('')
question_bank['tokenized_question_list'] = question_bank['question'].map(stem_tokenize)
question_bank['tokenized_question_str'] = question_bank['tokenized_question_list'].map(lambda x: ' '.join(x))
bm25_corpus = question_bank['tokenized_question_list'].tolist()
bm25 = BM25Okapi(bm25_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install git+https://github.com/Guzpenha/transformer_rankers.git
!wget https://raw.githubusercontent.com/Guzpenha/transformer_rankers/master/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/Guzpenha/transformer_rankers.git to /tmp/pip-req-build-gf8czg53
  Running command git clone --filter=blob:none --quiet https://github.com/Guzpenha/transformer_rankers.git /tmp/pip-req-build-gf8czg53
  Resolved https://github.com/Guzpenha/transformer_rankers.git to commit 637999ede0c1ae4389ccef5ac09337a5a2674ed7
  Preparing metadata (setup.py) ... done
  Created wheel for transformer-rankers: filename=transformer_rankers-0.0.1-py3-none-any.whl size=81331 sha256=70f8190bbf24abc9a24afd9fe9294b3594002d085db1f217c0c7fcadfa01a2bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-iv704_s0/wheels/9a/92/cc/b64666b3f23c86025c5af2d2ffd21f664d62b16ba40261561b
Successfully built transformer-rankers
--2023-04-15 07:00:26--  https://raw.githubusercontent.com/Guzpenha/transformer_rankers/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)..

In [ ]:
!pip install pytrec_eval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp39-cp39-linux_x86_64.whl size=293203 sha256=70a9926cb4ec5e8f56656a94b39d2c14b98b826e33473d1bf4052ea69d5614f5
  Stored in directory: /root/.cache/pip/wheels/e9/91/35/6059501bca98e27e0b4f91ecaaff86c95ca7f4919ff22f0d54
Successfully built pytrec_eval


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=1c5a7f9feafcce4c25990cad245afc07c12112f5e6ed429d6a1a3eac4754d6a3
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [ ]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=1eaf7fdb21911b1d8c3abceb132c67654ef920ef4b3c9d54a2598c028601851d
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [ ]:
!pip install faiss-cpu --no-cache

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 165.4 MB/s eta 0:00:00


In [ ]:
!apt-get install maven -qq
!git clone --recurse-submodules https://github.com/castorini/anserini.git
!cd anserini; mvn clean package appassembler:assemble -DskipTests -Dmaven.javadoc.skip=true
!ls anserini/target/appassembler/bin/

Extracting templates from packages: 100%
Selecting previously unselected package libapache-pom-java.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack .../00-libapache-pom-java_18-1_all.deb ...
Unpacking libapache-pom-java (18-1) ...
Selecting previously unselected package libatinject-jsr330-api-java.
Preparing to unpack .../01-libatinject-jsr330-api-java_1.0+ds1-5_all.deb ...
Unpacking libatinject-jsr330-api-java (1.0+ds1-5) ...
Selecting previously unselected package libgeronimo-interceptor-3.0-spec-java.
Preparing to unpack .../02-libgeronimo-interceptor-3.0-spec-java_1.0.1-4fakesync_all.deb ...
Unpacking libgeronimo-interceptor-3.0-spec-java (1.0.1-4fakesync) ...
Selecting previously unselected package libcdi-api-java.
Preparing to unpack .../03-libcdi-api-java_1.2-2_all.deb ...
Unpacking libcdi-api-java (1.2-2) ...
Selecting previously unselected package libcommons-cli-java.
Preparing to unpack .../04-libcommons-cli-java_1.4-1_all.deb ...

In [ ]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.5 MB/s eta 0:00:00


In [ ]:
from transformer_rankers.trainers import transformer_trainer
from transformer_rankers.datasets import dataset
from transformer_rankers.negative_samplers import negative_sampling
from transformer_rankers.eval import results_analyses_tools

from transformers import RobertaTokenizer, RobertaForSequenceClassification

import logging
import os
import sys
import torch
import random
import numpy as np

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

logging.basicConfig(
  level=logging.INFO,
  format="%(asctime)s [%(levelname)s] %(message)s",
  handlers=[
      logging.StreamHandler(sys.stdout)
  ]
)

#The combination of query and question are not that big.
max_seq_len = 50

#Lets use an almost balanced amount of positive and negative samples during training.
average_relevant_per_query = train.groupby("query").count().mean().values[0]

#Instantiate BM25 negative sampler.
# ns_train = negative_sampling.BM25NegativeSamplerPyserini(list(question_bank["question"].values[1:]), int(average_relevant_per_query) , 
#                     "/content/data/clariq/anserini_train/", -1, "./anserini/")
# ns_val = negative_sampling.BM25NegativeSamplerPyserini(list(question_bank["question"].values[1:]), int(average_relevant_per_query), 
#                     "/content/data/clariq/anserini_train/", -1, "./anserini/")

# We could also use random sampling which does not require Anserini.
ns_train = negative_sampling.RandomNegativeSampler(list(question_bank["question"].values[1:]), int(average_relevant_per_query))
ns_val = negative_sampling.RandomNegativeSampler(list(question_bank["question"].values[1:]), int(average_relevant_per_query))

#Create the loaders for the dataset, with the respective negative samplers
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
dataloader = dataset.QueryDocumentDataLoader(train_df=train,
                    val_df=valid, test_df=valid,
                    tokenizer=tokenizer, negative_sampler_train=ns_train,
                    negative_sampler_val=ns_val, task_type='classification',
                    train_batch_size=12, val_batch_size=12, max_seq_len=max_seq_len,
                    sample_data=-1, cache_path="./data/clariq/")

#time.sleep(120)
print(dataloader.get_pytorch_dataloaders())
train_loader,val_loader,test_loader = dataloader.get_pytorch_dataloaders()

#Use BERT (any model that has SequenceClassification class from HuggingFace would work here)
model = RobertaForSequenceClassification.from_pretrained("roberta-base")
# #Instantiate trainer that handles fitting.
trainer = transformer_trainer.TransformerTrainer(model=model,
  train_loader=train_loader,
  val_loader=val_loader, test_loader=test_loader,
  num_ns_eval=int(average_relevant_per_query), task_type="classification", tokenizer=tokenizer,
  validate_every_epochs=1,num_validation_batches=-1,
  num_epochs=3, lr=5e-6, sacred_ex=None)

#Train (our validation eval uses the NS sampling procedure)
trainer.fit()

  0%|          | 0/1 [00:00<?, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
100%|██████████| 1/1 [00:01<00:00,  1.62s/it]


(<torch.utils.data.dataloader.DataLoader object at 0x7f0b554bf340>, <torch.utils.data.dataloader.DataLoader object at 0x7f0b280abdf0>, <torch.utils.data.dataloader.DataLoader object at 0x7f0b28023190>)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [ ]:
from transformers.data.data_collator import DefaultDataCollator
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification

class SimpleDataset(Dataset):
    def __init__(self, features):
        self.features = features
    def __len__(self):
        return len(self.features)
    def __getitem__(self, index):
        return self.features[index]

#Lets not use the null document for no question.
all_documents = list(question_bank["question"].values[1:])
examples = []
for tid in dev['topic_id'].unique():
    query = dev.loc[dev['topic_id']==tid, 'initial_request'].tolist()[0]
    for doc in all_documents:
      examples.append((query, doc))

max_seq_len = 50
batch_encoding = tokenizer.batch_encode_plus(examples,max_length=max_seq_len,pad_to_max_length=True)
features = []
for i in range(len(examples)):
    inputs = {k: batch_encoding[k][i] for k in batch_encoding}
    feature = InputFeatures(**inputs, label=0)
    features.append(feature)

dataset = SimpleDataset(features)
collator = DefaultDataCollator()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs w

In [ ]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=False, collate_fn=collator)
from transformer_rankers.utils import utils
logits, _, softmax_output = trainer.predict(dataloader)
softmax_output_by_query = utils.acumulate_list(softmax_output[0], len(all_documents))

Predicting: 100%|██████████| 12313/12313 [09:39<00:00, 21.24it/s]


In [ ]:
import numpy as np
run_file_path = 'dev_RoBERTa-ranker'
all_doc_ids = np.array(question_bank["question_id"].values[1:])
with open(run_file_path, 'w') as fo:
  for tid_idx, tid in enumerate(dev['topic_id'].unique()):
    all_documents_scores = np.array(softmax_output_by_query[tid_idx])
    top_30_scores_idx = (-all_documents_scores).argsort()[:30]  
    preds = all_doc_ids[top_30_scores_idx]
    for i, qid in enumerate(preds):    
      fo.write('{} 0 {} {} {} RoBERTa-ranker\n'.format(tid, qid, i, len(preds)-i))

In [ ]:
    from statistics import mean
    topic_df = dev
    run_file = run_file_path
    topic_question_set_dict = topic_df.groupby('topic_id')['question_id'].agg(set).to_dict()
    run_df = pd.read_csv(run_file, sep=' ', header=None)
    run_df = run_df.sort_values(by=[0, 4], ascending=False).drop_duplicates(subset=[0, 4], keep='first')
    run_question_set_list = run_df.groupby(0)[2].agg(list).to_dict()
    topk_list = [5, 10, 20, 30]
    recall_score_dict = {}
    for topk in topk_list:
        metric_name = 'Recall{}'.format(topk)
        recall_score_dict[metric_name] = {}
        for tid in topic_question_set_dict:
            rec = len(set(run_question_set_list[tid][:topk]) & topic_question_set_dict[tid]) / len(
                    topic_question_set_dict[tid])
            recall_score_dict[metric_name][tid] = rec


    mean_performance = {}
    for metric in recall_score_dict:
        mean_performance[metric] = mean(recall_score_dict[metric][k] for k in recall_score_dict[metric])

    for metric in recall_score_dict:
        print('{}: {}'.format(metric, mean_performance[metric]))

Recall5: 0.34765917527604834
Recall10: 0.6131311933680355
Recall20: 0.738945681511471
Recall30: 0.7589843074881775


In [ ]:
  import pickle
  topic_df = dev
  # print(topic_df)
  context_array = topic_df['facet_id'].values

  eval_file_path = '/content/ClariQ/data/single_turn_train_eval.pkl'
  with open(eval_file_path, 'rb') as fi:
            eval_dict = pickle.load(fi)

  new_eval_dict = {}
  for metric in eval_dict:
      new_eval_dict[metric] = {}
      for fid in eval_dict[metric]:
          if fid in context_array:
                new_eval_dict[metric][fid] = eval_dict[metric][fid]

  eval_dict = new_eval_dict
  
  run_df = pd.read_csv(run_file, sep=' ', header=None).fillna('')
  run_df = run_df.sort_values(by=4).drop_duplicates(subset=[0], keep='last')  # we only keep the top ranked question.
  run_dict = run_df.set_index(0)[2].to_dict()
  
  facet_to_topic_dict = topic_df.set_index('facet_id')['topic_id'].to_dict()

  performance_dict = {}
  for metric in eval_dict:
        performance_dict[metric] = {}
        for context_id in eval_dict[metric]:
            try:
              selected_q = run_dict[facet_to_topic_dict[context_id]]
              selected_q = 'MIN' if selected_q == 'MAX' else selected_q 
              
              try:
                performance_dict[metric][context_id] = eval_dict[metric][context_id][selected_q]['with_answer']
              except KeyError:  # if question is not among candidate question, we consider it equal to minimum performance.
                performance_dict[metric][context_id] = eval_dict[metric][context_id]['MIN']['with_answer']
            except KeyError:  # if there is no prediction provided for a facet, we consider performance 0.
              performance_dict[metric][context_id] = 0.
  

    # compute the mean performance per metric and print
  mean_performance = {}
  for metric in performance_dict:
        mean_performance[metric] = mean(performance_dict[metric][k] for k in performance_dict[metric])
  for metric in performance_dict:
      print('{}: {}'.format(metric, mean_performance[metric]))

NDCG1: 0.22239583333333332
NDCG3: 0.17758933294551008
NDCG5: 0.17040349139826586
NDCG10: 0.16338321662769803
NDCG20: 0.14941041389164078
P1: 0.275
P3: 0.20208333333333334
P5: 0.1875
P10: 0.17
P20: 0.134375
MRR100: 0.3497495322070059
